In [1]:
!pip install transformers

In [2]:

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys   

In [2]:
train_path = "/kaggle/input/data-capec-multilabel/data_capec_multilabel.csv"


In [4]:
df = pd.read_csv(train_path)
# df = df.head(90000)
def create_text(method, request, body):
    if method == 'GET':
        return str(method) + ' ' + str(request)
    return str(method) + ' ' + str(request) + ' ' + str(body)

df['text'] = df.apply(lambda x: create_text(x['request_http_method'], x['request_http_request'], x['request_body']), axis = 1)

/tmp/ipykernel_32/2462438830.py:1: DtypeWarning: Columns (9,11,12,13,14,15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(train_path)


In [14]:
target_list = ["000 - Normal", '126 - Path Traversal', 
               '242 - Code Injection', '274 - HTTP Verb Tampering', 
               '66 - SQL Injection', '88 - OS Command Injection']

In [7]:
label = [df[i] for i in target_list]
print(type(label[0]))

<class 'pandas.core.series.Series'>


In [8]:
frame = {
    'text' : df['text'],
    '000 - Normal' : label[0],
    '126 - Path Traversal' : label[1],
    "242 - Code Injection" : label[2],
    '274 - HTTP Verb Tampering' : label[3],
    '66 - SQL Injection' : label[4],
    '88 - OS Command Injection' : label[5],
}
df = pd.DataFrame(frame)

In [19]:
# df["text"][2962]

In [9]:
df1 = df.head(200000)

In [10]:
df.columns

Index(['text', '000 - Normal', '126 - Path Traversal', '242 - Code Injection',
       '274 - HTTP Verb Tampering', '66 - SQL Injection',
       '88 - OS Command Injection'],
      dtype='object')

In [11]:
result_df = pd.DataFrame()
for label in target_list:
    result_df = pd.concat([result_df, df[df[label] == 1].head(15000)], ignore_index=True)
print(result_df)

                                                    text  000 - Normal  \
0                                                  GET /             1   
1      GET /blog/index.php/2020/04/04/voluptatum-repr...             1   
2                               GET /blog/xmlrpc.php?rsd             1   
3                                                  GET /             1   
4      GET /blog/index.php/2020/04/04/nihil-tenetur-e...             1   
...                                                  ...           ...   
72914              GET /blog/?p=%24%7B300661%2B757728%7D             0   
72915              GET /blog/?s=%24%7B263765%2B216301%7D             0   
72916  POST /cgi-bin/ViewLog.asp remote_submit_Flag=1...             0   
72917  POST /cgi-bin/ViewLog.asp  remote_submit_Flag=...             0   
72918  GET /cgi-bin/kerbynet?Section=NoAuthREQ&Action...             0   

       126 - Path Traversal  242 - Code Injection  274 - HTTP Verb Tampering  \
0                         0    

In [13]:
result_df.to_csv("data_raw.csv")

In [10]:
# df1 = pd.read_csv(train_path)
df[df[target_list] == 1].count()

text                              0
000 - Normal                 525195
126 - Path Traversal          20992
242 - Code Injection          15827
274 - HTTP Verb Tampering      5437
66 - SQL Injection           250311
88 - OS Command Injection      7482
dtype: int64

In [17]:
# df1 = pd.read_csv(train_path)
result_df[result_df[target_list] == 1].count()

text                             0
000 - Normal                 15000
126 - Path Traversal         17944
242 - Code Injection         16597
274 - HTTP Verb Tampering     5437
66 - SQL Injection           16086
88 - OS Command Injection     9319
dtype: int64

In [14]:
result_df.reset_index(drop=True, inplace=True)
result_df

,text,000 - Normal,126 - Path Traversal,242 - Code Injection,274 - HTTP Verb Tampering,66 - SQL Injection,88 - OS Command Injection
0,GET /,1,0,0,0,0,0
1,GET /blog/index.php/2020/04/04/voluptatum-repr...,1,0,0,0,0,0
2,GET /blog/xmlrpc.php?rsd,1,0,0,0,0,0
3,GET /,1,0,0,0,0,0
4,GET /blog/index.php/2020/04/04/nihil-tenetur-e...,1,0,0,0,0,0
...,...,...,...,...,...,...,...
72914,GET /blog/?p=%24%7B300661%2B757728%7D,0,0,0,0,0,1
72915,GET /blog/?s=%24%7B263765%2B216301%7D,0,0,0,0,0,1
72916,POST /cgi-bin/ViewLog.asp remote_submit_Flag=1...,0,0,0,0,0,1
72917,POST /cgi-bin/ViewLog.asp remote_submit_Flag=...,0,0,0,0,0,1


In [15]:
train, validate, test = np.split(result_df.sample(frac=1, random_state=42),[int(.6*len(result_df)), int(.8*len(result_df))])

In [16]:
train[train[target_list] == 1].count()

text                             0
000 - Normal                  9004
126 - Path Traversal         10671
242 - Code Injection         10134
274 - HTTP Verb Tampering     3216
66 - SQL Injection            9685
88 - OS Command Injection     5583
dtype: int64

In [17]:
validate[validate[target_list] == 1].count()

text                            0
000 - Normal                 2936
126 - Path Traversal         3615
242 - Code Injection         3260
274 - HTTP Verb Tampering    1111
66 - SQL Injection           3242
88 - OS Command Injection    1915
dtype: int64

In [18]:
test[test[target_list] == 1].count()

text                            0
000 - Normal                 3060
126 - Path Traversal         3658
242 - Code Injection         3203
274 - HTTP Verb Tampering    1110
66 - SQL Injection           3159
88 - OS Command Injection    1821
dtype: int64

In [10]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import BertTokenizer, BertModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')


In [12]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [15]:
test = pd.read_csv('/kaggle/input/test-data/test.csv')
test = test.reset_index(drop=True)
test = CustomDataset(test, tokenizer, MAX_LEN)
test_data_loader = torch.utils.data.DataLoader(test, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [33]:
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
test = test.reset_index(drop=True)

train = CustomDataset(train, tokenizer, MAX_LEN)
validate= CustomDataset(validate, tokenizer, MAX_LEN)
test = CustomDataset(test, tokenizer, MAX_LEN)

In [34]:
train_data_loader = torch.utils.data.DataLoader(train, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(validate, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
test_data_loader = torch.utils.data.DataLoader(test, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [36]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [8]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = AutoModel.from_pretrained('jackaduma/SecBERT')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 6)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()

model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [39]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [40]:
val_targets=[]
val_outputs=[]

In [41]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
   
 
    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0
        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch', batch_idx)
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            #if batch_idx%5000==0:
             #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print('before loss data in training', loss.item(), train_loss)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
            #print('after loss data in training', loss.item(), train_loss)
            
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

        print('############# Epoch {}: Validation End     #############'.format(epoch))
        # calculate average losses
        #print('before cal avg train loss', train_loss)
        if len(training_loader) > 0:
            train_loss = train_loss/len(training_loader)
        if len(validation_loader) > 0:
            valid_loss = valid_loss/len(validation_loader)
        # print training/validation statistics 
        print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

        print('############# Epoch {}  Done   #############\n'.format(epoch))

        return model

In [42]:
ckpt_path = "/kaggle/working/curr_ckpt"
best_model_path = "/kaggle/working/best_model.pt"

In [43]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000014 	Average Validation Loss: 0.000034
Validation loss decreased (inf --> 0.000034).  Saving model ...
############# Epoch 2  Done   #############



In [21]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

In [22]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
#     y_true1 = y_true.to_numpy()
#     y_pred1 = y_pred.toarray()
    acc_list = []
    
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [19]:
# from sklearn.metrics import hamming_loss,classification_report
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score, hamming_loss

In [18]:
def save_classification(y_test, y_pred, labels):
    if isinstance(y_pred, np.ndarray) == False:
        y_pred = y_pred.toarray()

    def accuracy(y_true, y_pred):
        temp = 0
        for i in range(y_true.shape[0]):
            numerator = sum(np.logical_and(y_true[i], y_pred[i]))
            denominator = sum(np.logical_or(y_true[i], y_pred[i]))
            if denominator != 0:
                temp += numerator / denominator
        return temp / y_true.shape[0]

    out = classification_report(y_test,y_pred, output_dict=True, target_names=labels)
    total_support = out['samples avg']['support']

    mr = accuracy_score(y_test, y_pred)
    acc = accuracy(y_test,y_pred)
    hm = hamming_loss(y_test, y_pred)

    out['Exact Match Ratio'] = {'precision': mr, 'recall': mr, 'f1-score': mr, 'support': total_support}
    out['Hamming Loss'] = {'precision': hm, 'recall': hm, 'f1-score': hm, 'support': total_support}
    out['Accuracy'] = {'precision': acc, 'recall': acc, 'f1-score': acc, 'support': total_support}
    out_df = pd.DataFrame(out).transpose()
    print(out_df)

    

    return out_df

In [17]:
def predict(testing_loader, model):
    print("\nPredicting...")
    # deactivate dropout layers
    model.eval()

    # empty list to save the model predictions
    total_preds = []
    total_labels = []
    # iterate over batches
    for step, batch in enumerate(testing_loader):
        # push the batch to gpu
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        targets = batch['targets'].to(device)

        # deactivate autograd
        with torch.no_grad():
            # model predictions
            output = model(ids, mask, token_type_ids)
            final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
            
            total_preds += list(final_output)
            total_labels += targets.tolist()

    return total_labels, total_preds

In [49]:
total_labels, total_preds = predict(test_data_loader, trained_model)



Predicting...


In [50]:
save_classification(y_test=np.array(np.nan_to_num(total_labels)), y_pred=np.array(np.round(total_preds)), labels=target_list)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                           precision    recall  f1-score  support
000 - Normal                0.960038  0.997059  0.978198   3060.0
126 - Path Traversal        0.998334  0.982777  0.990495   3658.0
242 - Code Injection        0.999684  0.986575  0.993086   3203.0
274 - HTTP Verb Tampering   1.000000  1.000000  1.000000   1110.0
66 - SQL Injection          0.998715  0.984172  0.991390   3159.0
88 - OS Command Injection   0.998853  0.956068  0.976992   1821.0
micro avg                   0.991198  0.984698  0.987937  16011.0
macro avg                   0.992604  0.984442  0.988360  16011.0
weighted avg                0.991534  0.984698  0.987963  16011.0
samples avg                 0.986606  0.986423  0.986455  16011.0
Exact Match Ratio           0.985943  0.985943  0.985943  16011.0
Hamming Loss                0.004400  0.004400  0.004400  16011.0
Accuracy                    0.986332  0.986332  0.986332  16011.0


,precision,recall,f1-score,support
000 - Normal,0.960038,0.997059,0.978198,3060.0
126 - Path Traversal,0.998334,0.982777,0.990495,3658.0
242 - Code Injection,0.999684,0.986575,0.993086,3203.0
274 - HTTP Verb Tampering,1.000000,1.000000,1.000000,1110.0
66 - SQL Injection,0.998715,0.984172,0.991390,3159.0
88 - OS Command Injection,0.998853,0.956068,0.976992,1821.0
micro avg,0.991198,0.984698,0.987937,16011.0
macro avg,0.992604,0.984442,0.988360,16011.0
weighted avg,0.991534,0.984698,0.987963,16011.0
samples avg,0.986606,0.986423,0.986455,16011.0


In [ ]:
# np.nan_to_num(total_labels)

In [ ]:
# np.round(total_preds)

In [9]:
checkpoint = torch.load('/kaggle/input/secbert_test/pytorch/secbert_test/1/best_model_raw_data.pt')
model = BERTClass()
# model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))
model.load_state_dict(checkpoint['state_dict'])
model.eval()

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [52]:
# %cd /kaggle/working
from IPython.display import FileLink
FileLink(r'best_model.pt')

/kaggle/working/best_model.pt

In [20]:
total_labels, total_preds = predict(test_data_loader, model.to(device))



Predicting...


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                           precision    recall  f1-score  support
000 - Normal                0.944515  0.993994  0.968623   2997.0
126 - Path Traversal        0.995153  0.968249  0.981517   3181.0
242 - Code Injection        0.985434  0.995203  0.990294   3127.0
274 - HTTP Verb Tampering   1.000000  1.000000  1.000000   1111.0
66 - SQL Injection          0.998825  0.815276  0.897765   3129.0
88 - OS Command Injection   0.828143  0.981545  0.898342   1463.0
micro avg                   0.963731  0.950760  0.957201  15008.0
macro avg                   0.958678  0.959045  0.956090  15008.0
weighted avg                0.967860  0.950760  0.956570  15008.0
samples avg                 0.947026  0.947551  0.947143  15008.0
Exact Match Ratio           0.945938  0.945938  0.945938  15008.0
Hamming Loss                0.015248  0.015248  0.015248  15008.0
Accuracy                    0.946846  0.946846  0.946846  15008.0


,precision,recall,f1-score,support
000 - Normal,0.944515,0.993994,0.968623,2997.0
126 - Path Traversal,0.995153,0.968249,0.981517,3181.0
242 - Code Injection,0.985434,0.995203,0.990294,3127.0
274 - HTTP Verb Tampering,1.000000,1.000000,1.000000,1111.0
66 - SQL Injection,0.998825,0.815276,0.897765,3129.0
88 - OS Command Injection,0.828143,0.981545,0.898342,1463.0
micro avg,0.963731,0.950760,0.957201,15008.0
macro avg,0.958678,0.959045,0.956090,15008.0
weighted avg,0.967860,0.950760,0.956570,15008.0
samples avg,0.947026,0.947551,0.947143,15008.0


In [ ]:
save_classification(y_test=np.array(np.nan_to_num(total_labels)), y_pred=np.array(np.round(total_preds)), labels=target_list)